In [51]:
import fml_lib
import numpy as np
import pandas as pd
import scipy.stats as ss
from math import sqrt, exp, log
import matplotlib.pyplot as plt

Exercise 1: A portfolio manager intends to launch a strategy that targets an annualized SR of 2. Bets have a precision rate of 60%, with weekly frequency. The exit conditions are 2% for profit-taking, and –2% for stop-loss.


In [52]:
def mixGaussians(mu1,mu2,sigma1,sigma2,prob1,nObs):
    # Random draws from a mixture of gaussians
    ret1=np.random.normal(mu1,sigma1,size=int(nObs*prob1))
    ret2=np.random.normal(mu2,sigma2,size=int(nObs)-ret1.shape[0])
    ret=np.append(ret1,ret2,axis=0)
    np.random.shuffle(ret)
    return ret

def binHR(sl,pt,freq,tSR):
    '''
    Given a trading rule characterized by the parameters {sl,pt,freq},
    what's the min precision p required to achieve a Sharpe ratio tSR?
    1) Inputs
    
    sl: stop loss threshold
    pt: profit taking threshold
    freq: number of bets per year
    tSR: target annual Sharpe ratio
    2) Output
    p: the min precision rate p required to achieve tSR
    '''
    a=(freq+tSR**2)*(pt-sl)**2
    b=(2*freq*sl-tSR**2*(pt-sl))*(pt-sl)
    c=freq*sl**2
    p=(-b+(b**2-4*a*c)**.5)/(2.*a)
    return p

def binSR(sl,pt,p,freq):
    '''
    Given a trading rule characterized by the parameters {sl,pt,p,freq},
    what's estimate sharpe ratio?\
    1) Inputs
    sl: stop loss threshold
    pt: profit taking threshold
    p: precision rate p
    freq: number of bets per year
    2) Output
    SR: estimated sharpe ratio

    '''

    sr = np.sqrt(freq)*((pt-sl)*p+sl)/((pt-sl)*(np.sqrt(p*(1-p)))) 
    return sr


def binFreq(sl,pt,p,tSR):
    '''
    Given a trading rule characterized by the parameters {sl,pt,freq},
    what's the number of bets/year needed to achieve a Sharpe ratio
    tSR with precision rate p?
    Note: Equation with radicals, check for extraneous solution.
    1) Inputs
    sl: stop loss threshold
    pt: profit taking threshold
    p: precision rate p
    tSR: target annual Sharpe ratio
    2) Output
    freq: number of bets per year needed

    '''

    freq=(tSR*(pt-sl))**2*p*(1-p)/((pt-sl)*p+sl)**2 # possible extraneous
    if not np.isclose(binSR(sl,pt,p,freq),tSR):
        return
    return freq

def binPT(sl,p,freq,tSR):
    pt = (sl)/(-p+(np.sqrt(p*(1-p))*tSR/np.sqrt(freq))) + sl
    return pt

def binSL(pt,p,freq,tSR):
    sl = pt/(1+(1/(-p+np.sqrt(p*(1-p))*tSR/np.sqrt(freq))))
    return sl


def probFailure(ret,freq,tSR):
    # Derive probability that strategy may fail
    rPos,rNeg=ret[ret>0].mean(),ret[ret<=0].mean()
    p=ret[ret>0].shape[0]/float(ret.shape[0])
    thresP=binHR(rNeg,rPos,freq,tSR)
    risk=ss.norm.cdf(thresP,p,p*(1-p)) # approximation to bootstrap
    return risk

In [53]:
pt,sl = 0.02,-0.02 # profit taking and stop loss conditions
n = 52 # frequency is weekly, so there will be 52 trades per year
p = 0.6 # precision is 60%

min_p = binHR(sl, pt, n, 0)
print(f"Minimum precision required for a viable strategy is {min_p}")

Minimum precision required for a viable strategy is 0.5


1.a: Is this strategy viable?
A: Yes, the minimum precision required to not lose money is 0.5 but the strategy has a precision of 0.6

1.b: Ceteris paribus, what is the required precision rate that would make the
strategy profitable?
A: Any precision rate above 0.5 would make the strategy profitable

1.c:  For what betting frequency is the target achievable?
A: See below

In [54]:
tSR = 2
min_freq = binFreq(sl,pt,p,2)
print(f"Minimum frequency needed for a target Sharpe ratio of {tSR} is {min_freq}")

Minimum frequency needed for a target Sharpe ratio of 2 is 96.0


1.d: For what profit taking threshold is the target achievable?
A: See below

In [55]:
min_pt = binPT(sl,p,n,tSR)
print(f"The minimum profit taking threshold for the target SR {tSR} is {min_pt}")

The minimum profit taking threshold for the target SR 2 is 0.023091676456151216


1.e: What would be an alternative stop loss?
A: See below

In [56]:
min_sl = binSL(pt,p,n,tSR)
print(f"The minimum stop loss threshold for the target SR {tSR} is {min_sl}")

The minimum stop loss threshold for the target SR 2 is -0.017322258986243816
